In [5]:
import json
import re

import pandas as pd
import numpy as np

from lxml import etree
import requests
import os

from pykml import parser
# from geopy.distance import geodesic

In [8]:
def onemap_search(placename):
    onemapSearchUrl = "https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y"
    res = requests.get(onemapSearchUrl.format(placename))
    if res.status_code != 200:
        return None
    try:
        res = res.json()
    except Exception as e:
        print(placename, e)
    if res['found'] == 0:
        return None
    return res['results'][0]

In [11]:
onemap_search('5 science park')

{'ADDRESS': '5 SCIENCE PARK DRIVE SINGAPORE 118265',
 'BLK_NO': '5',
 'BUILDING': 'NIL',
 'LATITUDE': '1.29207905014866',
 'LONGITUDE': '103.786710614115',
 'LONGTITUDE': '103.786710614115',
 'POSTAL': '118265',
 'ROAD_NAME': 'SCIENCE PARK DRIVE',
 'SEARCHVAL': '5 SCIENCE PARK DRIVE SINGAPORE 118265',
 'X': '22812.9348640535',
 'Y': '30497.1033341698'}

In [14]:
onemap_search('118265')

{'ADDRESS': '5 SCIENCE PARK DRIVE SINGAPORE 118265',
 'BLK_NO': '5',
 'BUILDING': 'NIL',
 'LATITUDE': '1.29207905014866',
 'LONGITUDE': '103.786710614115',
 'LONGTITUDE': '103.786710614115',
 'POSTAL': '118265',
 'ROAD_NAME': 'SCIENCE PARK DRIVE',
 'SEARCHVAL': '5 SCIENCE PARK DRIVE SINGAPORE 118265',
 'X': '22812.9348640535',
 'Y': '30497.1033341698'}

###### process hawker center

In [32]:
with open('./data/_raw/hawker-centres/hawker-centres-geojson.geojson') as f:
    hawker = json.load(f)

In [33]:
hawker.keys()

dict_keys(['crs', 'type', 'features'])

In [52]:
name_re = re.compile(r'<th>NAME</th> <td>(.+?)</td>')
LONGITUDE = []
LATITUDE = []
NAMES = []
for feature in hawker['features']:
#     print(feature['geometry']['coordinates'])
#     print(re.findall(name_re, feature['properties']['Description'])[0])
    LONGITUDE.append(feature['geometry']['coordinates'][0])
    LATITUDE.append(feature['geometry']['coordinates'][1])
    NAMES.append(re.findall(name_re, feature['properties']['Description'])[0])
df_hawker = pd.DataFrame(data={'NAME':NAMES, 'LONGITUDE':LONGITUDE, 'LATITUDE':LATITUDE})

In [53]:
df_hawker.head()

,LATITUDE,LONGITUDE,NAME
0,1.306206,103.850531,Buffalo Road Blk 665 (Tekka Centre/Zhu Jiao Ma...
1,1.283693,103.816992,Bukit Merah Central Blk 163 (Bukit Merah Centr...
2,1.286298,103.804489,Bukit Merah Lane 1 Blk 120 (Alexandra Village ...
3,1.301001,103.854212,Queen Street Blk 270 (Albert Centre)
4,1.335231,103.857036,Toa Payoh Lorong 7 Blk 22 (Kim Keat Palm Marke...


In [54]:
df_hawker.to_csv("./data/_processed/TBL_HAWKER_ADDR_INFO.csv",index=False)

In [177]:
df_hawker = pd.read_csv("./data/_processed/TBL_HAWKER_ADDR_INFO.csv")

In [182]:
df_hawker.head()

,LATITUDE,LONGITUDE,NAME
0,1.306206,103.850531,Buffalo Road Blk 665 (Tekka Centre/Zhu Jiao Ma...
1,1.283693,103.816992,Bukit Merah Central Blk 163 (Bukit Merah Centr...
2,1.286298,103.804489,Bukit Merah Lane 1 Blk 120 (Alexandra Village ...
3,1.301001,103.854212,Queen Street Blk 270 (Albert Centre)
4,1.335231,103.857036,Toa Payoh Lorong 7 Blk 22 (Kim Keat Palm Marke...


###### process MRT

In [108]:
df_mrt = pd.read_csv("./data/_raw/mrt-lrt-stations/mrt_lrt_data.csv")

In [109]:
df_mrt.head()

,NAME,TYPE,LATITUDE,LONGITUDE
0,Jurong East,MRT,1.333207,103.742308
1,Bukit Batok,MRT,1.349069,103.749596
2,Bukit Gombak,MRT,1.359043,103.751863
3,Choa Chu Kang,MRT,1.385417,103.744316
4,Yew Tee,MRT,1.397383,103.747523


In [110]:
df_mrt.to_csv("./data/_processed/TBL_MRT_LRT_ADDR_INFO.csv",index=False)

In [184]:
df_mrt = pd.read_csv("./data/_processed/TBL_MRT_LRT_ADDR_INFO.csv")

In [185]:
df_mrt.head()

,NAME,TYPE,LATITUDE,LONGITUDE
0,Jurong East,MRT,1.333207,103.742308
1,Bukit Batok,MRT,1.349069,103.749596
2,Bukit Gombak,MRT,1.359043,103.751863
3,Choa Chu Kang,MRT,1.385417,103.744316
4,Yew Tee,MRT,1.397383,103.747523


###### process Mall

In [75]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore')
html = res.content
subpage = etree.HTML(html)

In [103]:
malls = subpage.xpath('//*[@class="div-col"]/ul/li//text()')

In [104]:
len(malls)

175

In [143]:
malls = [m.split('(')[0] if "(" in m else m for m in malls]

In [144]:
df_mall = pd.DataFrame(data={'NAME':malls})

In [145]:
df_mall.head(2)

,NAME
0,100 AM
1,313@Somerset


In [149]:
df_mall['data'] = df_mall['NAME'].apply(onemap_search)

In [154]:
df_mall['BLOCK'] = df_mall['data'].apply(lambda x: x['BLK_NO'] if x != None else None)
df_mall['ADDRESS'] = df_mall['data'].apply(lambda x: x['ADDRESS'] if x != None else None)
df_mall['BUILDING'] = df_mall['data'].apply(lambda x: x['BUILDING'] if x != None else None)
df_mall['LATITUDE'] = df_mall['data'].apply(lambda x: x['LATITUDE'] if x != None else None)
df_mall['LONGITUDE'] = df_mall['data'].apply(lambda x: x['LONGITUDE'] if x != None else None)
df_mall['POSTAL_CODE'] = df_mall['data'].apply(lambda x: x['POSTAL'] if x != None else None)
df_mall['STREET_NAME'] = df_mall['data'].apply(lambda x: x['ROAD_NAME'] if x != None else None)
df_mall['X'] = df_mall['data'].apply(lambda x: x['X'] if x != None else None)
df_mall['Y'] = df_mall['data'].apply(lambda x: x['Y'] if x != None else None)

In [156]:
df_mall = df_mall.drop(columns=['data'])

In [188]:
df_mall.to_csv('./data/_processed/TBL_SHOPPING_MALL_ADDR_INFO.csv',index=False)

In [189]:
df_mall = pd.read_csv("./data/_processed/TBL_SHOPPING_MALL_ADDR_INFO.csv")

In [190]:
df_mall.head()

,NAME,ADDRESS,BLOCK,BUILDING,LATITUDE,LONGITUDE,POSTAL_CODE,STREET_NAME,X,Y
0,100 AM,100 TRAS STREET 100 AM SINGAPORE 079027,100,100 AM,1.274588,103.843471,079027,TRAS STREET,29129.855226,28563.012106
1,313@Somerset,313 ORCHARD ROAD UOB 313@SOMERSET SINGAPORE 23...,313,UOB 313@SOMERSET,1.301385,103.837684,238895,ORCHARD ROAD,28485.870911,31526.071401
2,Aperia,12 KALLANG AVENUE JOSIAH BABIES (APERIA) SINGA...,12,JOSIAH BABIES (APERIA),1.309742,103.864102,339511,KALLANG AVENUE,31425.866699,32450.201191
3,Balestier Hill Shopping Centre,1A THOMSON ROAD BALESTIER HILL SHOPPING CENTRE...,1A,BALESTIER HILL SHOPPING CENTRE,1.325761,103.843054,301001,THOMSON ROAD,29083.463125,34221.465438
4,Bugis Cube,470 NORTH BRIDGE ROAD BUGIS CUBE SINGAPORE 188735,470,BUGIS CUBE,1.298195,103.855656,188735,NORTH BRIDGE ROAD,30485.905724,31173.337430


###### process School

In [161]:
df_school = pd.read_csv("./data/_processed/TBL_SCHOOL_INFO.csv.gzip", sep="|", dtype={"POSTAL_CODE":'object'})

In [163]:
df_school.head(2)

,SCHOOL_NAME,URL_ADDRESS,ADDRESS,POSTAL_CODE,TELEPHONE_NO,TELEPHONE_NO_2,FAX_NO,FAX_NO_2,EMAIL_ADDRESS,MRT_DESC,...,SESSION_CODE,MAINLEVEL_CODE,SAP_IND,AUTONOMOUS_IND,GIFTED_IND,IP_IND,MOTHERTONGUE1_CODE,MOTHERTONGUE2_CODE,MOTHERTONGUE3_CODE,SPECIAL_SDP_OFFERED
0,ADMIRALTY PRIMARY SCHOOL,http://www.admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,NaN,63627512,NaN,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,FULL DAY,PRIMARY,0,0,0,0,Chinese,Malay,Tamil,"With globalization, it is important for our st..."
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,NaN,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,SINGLE SESSION,SECONDARY,0,0,0,0,Chinese,Malay,Tamil,Design Thinking is used as an anchor pedagogic...


In [164]:
df_school['data'] = df_school['ADDRESS'].apply(onemap_search)

In [170]:
df_school['BLOCK'] = df_school['data'].apply(lambda x: x['BLK_NO'] if x != None else None)
df_school['ADDRESS'] = df_school['data'].apply(lambda x: x['ADDRESS'] if x != None else None)
df_school['BUILDING'] = df_school['data'].apply(lambda x: x['BUILDING'] if x != None else None)
df_school['LATITUDE'] = df_school['data'].apply(lambda x: x['LATITUDE'] if x != None else None)
df_school['LONGITUDE'] = df_school['data'].apply(lambda x: x['LONGITUDE'] if x != None else None)
df_school['POSTAL_CODE'] = df_school['data'].apply(lambda x: x['POSTAL'] if x != None else None)
df_school['STREET_NAME'] = df_school['data'].apply(lambda x: x['ROAD_NAME'] if x != None else None)
df_school['X'] = df_school['data'].apply(lambda x: x['X'] if x != None else None)
df_school['Y'] = df_school['data'].apply(lambda x: x['Y'] if x != None else None)

In [173]:
df_school = df_school[['SCHOOL_NAME', 'ADDRESS', 'BLOCK', 'BUILDING',
       'LATITUDE', 'LONGITUDE', 'STREET_NAME', 'X', 'Y', 'POSTAL']]

In [194]:
df_school.rename(columns={'SCHOOL_NAME':'NAME'},inplace=True)

In [197]:
df_school.head(2)

,NAME,ADDRESS,BLOCK,BUILDING,LATITUDE,LONGITUDE,STREET_NAME,X,Y,POSTAL_CODE
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE ADMIRALTY PRIMARY SCHOOL S...,11,ADMIRALTY PRIMARY SCHOOL,1.44267961678131,103.800114351557,WOODLANDS CIRCLE,24304.8860053159,47149.7266234159,738907
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT ADMIRALTY SECONDARY SCHO...,31,ADMIRALTY SECONDARY SCHOOL,1.44589068910991,103.802398196596,WOODLANDS CRESCENT,24559.0472937012,47504.7873554379,737916


In [198]:
df_school.to_csv('./data/_processed/TBL_SCHOOL_ADDR_INFO.csv',index=False)

In [199]:
df_school = pd.read_csv("./data/_processed/TBL_SCHOOL_ADDR_INFO.csv")

In [200]:
df_school.head()

,NAME,ADDRESS,BLOCK,BUILDING,LATITUDE,LONGITUDE,STREET_NAME,X,Y,POSTAL_CODE
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE ADMIRALTY PRIMARY SCHOOL S...,11,ADMIRALTY PRIMARY SCHOOL,1.442680,103.800114,WOODLANDS CIRCLE,24304.886005,47149.726623,738907
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT ADMIRALTY SECONDARY SCHO...,31,ADMIRALTY SECONDARY SCHOOL,1.445891,103.802398,WOODLANDS CRESCENT,24559.047294,47504.787355,737916
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11 AHMAD IBRAHIM PRIMARY SCHO...,10,AHMAD IBRAHIM PRIMARY SCHOOL,1.433681,103.832924,YISHUN STREET 11,27956.121629,46154.700680,768643
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7 AHMAD IBRAHIM SECONDARY SC...,751,AHMAD IBRAHIM SECONDARY SCHOOL,1.436235,103.829986,YISHUN AVENUE 7,27629.184969,46437.102724,768928
4,AI TONG SCHOOL,100 BRIGHT HILL DRIVE AI TONG SCHOOL SINGAPORE...,100,AI TONG SCHOOL,1.360736,103.833076,BRIGHT HILL DRIVE,27972.965368,38088.829447,579646


###### compute distance from facility to HDB block

In [202]:
def nearest_distance(hdb_row, fac_df):
    hdb_lat, hdb_lon = hdb_row[['LATITUDE','LONGITUDE']]
    nearest_dist = np.inf
    for fac_lat, fac_lon in zip(fac_df['LATITUDE'], fac_df['LONGITUDE']):
        dist = geodesic((hdb_lat, hdb_lon), (fac_lat, fac_lon)).m
        if dist < nearest_dist:
            nearest_dist = dist
    return nearest_dist

In [203]:
df_hdb = pd.read_csv("./data/_processed/TBL_HDB_ADDR_INFO.csv.gzip", sep="|", dtype={"POSTAL_CODE":'object'})

In [204]:
df_hdb.head(2)

,TOWN,BLOCK,STREET_NAME,POSTAL_CODE,ADDRESS,BUILDING,LATITUDE,LONGITUDE,X,Y
0,KALLANG/WHAMPOA,1,BEACH RD,190001,1 BEACH ROAD BEACH ROAD GARDENS SINGAPORE 190001,BEACH ROAD GARDENS,1.303671,103.864479,31467.832743,31778.893997
1,BEDOK,1,BEDOK STH AVE 1,460001,1 BEDOK SOUTH AVENUE 1 SINGAPORE 460001,NIL,1.320852,103.933721,39173.812353,33678.855652


In [207]:
hawker_dist = df_hdb.apply(nearest_distance, axis=1, fac_df=df_hawker)

In [208]:
df_hdb_neighbor = pd.DataFrame({'ADDRESS': df_hdb['ADDRESS'], 'hawker_dist': hawker_dist})

In [213]:
df_hdb_neighbor['hawker_dist'].max()

2.8676263652034715